# How to run this file?
In this file, there are three cells: 'define functions', 'simulation', 'graphing' cell. 
- First, run the 'define functions' cell to define functions.
- After you run 'define functions' cell, you will see 'config parameters' section explains the parameters you can change in the next cell ('simulation' cell).
- After you modify parameters, run the 'simulation' cell. You will see '-------Simulation Start----------',..., until'----------Simulation Finished---------', meaning simulation completed. The processing time depends on the game length you select.
- Finally, run the 'graphing' cell. The result shows dynamics graphs over periods. The top left graph shows payoffs for each strategies over periods and green triangle shows which player gets the chance to move; The top right graph shows cdf; the bottom graph shows strategies over periods. 

In [1]:
#########################################################
########## run this cell to define functions ############
############### 'define functions' cell #################
#########################################################


########## configuration ############
import math

def sim_config_init(game_type= 'fear', lgr=(10,1.1,0.5), sampling=None, purification=None,
                    trembling=0., theta=None, bandwidth=None, asynchronous=True,
                    num_bots=20, game_length=1000, xrange=(2,10), move_percent = 0.1,
                    ):
    '''
    Specify simulation configuration parameters
    :param game_type: string, game type
    :param lgr: tuple, (lambda, gamma, rho), default to (10,1.1,0.5)
    :param sampling: int, number of players to sample (give each player an array of random other players to sample), default to None
    :param purification: idiosyncratic shifts of perceived landscape
    :param trembling: float, trembling range, default to 0, trembles in jump destination
    :param theta: float, constant for calculating player move chance, default to 0.2
    :param bandwidth: float, smoothing bandwidth, default to None
    :param asynchronous: boolean, game synchronicity, False means all players decide their moves based on the previous tick, True means players see moves as they happen, default to False
    :param num_bots: int, number of bots, default to 20
    :param game_length: int, game length, default to 1000
    :param xrange: tuple, strategy range
    :return: config, dictionary containing all simulation configuration parameters
    '''

    config = {}

    # set to -1 to disable
    config['sampling'] = sampling

    # constant e in purification specs
    # set to -1 to disable
    config['purification'] = purification

    # trembling range
    # set to 0 to have no effect
    config['trembling'] = trembling

    # constant for calculating player move chance
    config['theta'] = theta

    # smoothing bandwidth
    # set to -1 to disable
    config['bandwidth'] = bandwidth

    # game type
    config['game_type'] = game_type

    # lambda/gamma/rho params
    config['lambda'] = lgr[0]
    config['gamma'] = lgr[1]
    config['rho'] = lgr[2]

    # based on gamma and rho, override game type
    if (config['game_type'] != 'other') and (config['gamma'] >= (config['rho'] + 4/3)):
        config['game_type'] = 'greed'
    elif (config['game_type'] != 'other') and (config['gamma'] <= (config['rho'] + 2/3)):
        config['game_type'] = 'fear'

    # rush range (MUST BE CORRECT IF STARTING AT CDF)
    if config['game_type'] == 'fear':
        config['cdfmin'] = max(0, round((config['lambda'] - math.sqrt(1+config['lambda']**2) * math.sqrt(1-(16*(1+config['rho'])*(config['gamma']-1))/((config['gamma'] +3*config['rho'])*(3*config['gamma'] +config['rho'])))),2))
        config['cdfmax'] = config['lambda']
    elif config['game_type'] == 'greed':
        config['cdfmin'] = config['lambda']
        config['cdfmax'] = config['lambda'] + math.sqrt(1+config['lambda']**2)/(math.sqrt(1+16*config['rho']*config['gamma']/((3*config['gamma']-3*config['rho']-2)*(config['gamma']-config['rho']+2))))

    # game synchronicity
    # False means all players decide their moves based on the previous tick
    # True means players see moves as they happen
    config['asynchronous'] = asynchronous

    # number of bots
    config['num_bots'] = num_bots

    # game length
    config['game_length'] = game_length

    # x bound
    if (config['game_type'] == 'fear') or (config['game_type'] == 'greed'):
        config['xmin'] = max(0 , config['cdfmin']-2)
        config['xmax'] = config['cdfmax'] + 3
    else:
        config['xmin'] = xrange[0]
        config['xmax'] = xrange[1]
        
    config['move_percent'] = move_percent

    return config

########## functions ############

# Returns the position in group for a given timing n (for a given n, how many strategies are less than n)
# optional parameter seed denotes which set of players to sample (-1 for no sampling)
def get_position(n, seed, strats, sample_sets, config):
    '''

    :param n: float, timing n
    :param seed: index of player i, default to none
    :param strats: array, current strategies
    :param sample_sets: ndarray, sample sets (None if sampling is none), give each player an array of random other players to sample
    :param config: dict, dictionary containing simulation parameters
    :return: float, position for timing n
    '''
    # positions are approximated more accurately by adding 0 to the value for greed and 1 for fear game
    if config['game_type'] == 'fear':
        pos = 1
    elif config['game_type'] == 'greed':
        pos = 0
    else:
        pos = 0.5
        
    samples = []
    # if using sampling, get the strategies of the sampled players
    if (seed is not None) and (config['sampling'] is not None):
        for samp in sample_sets[seed]:
            samples.append(strats[samp])
    # otherwise check all strategies
    else:
        samples = strats
    # compare to strategies to calculate position
    for strat in samples:
        if n > strat:
            pos = pos + 1
    return pos

# Returns ties at timing n
# optional parameter seed denotes which set of players to sample (-1 for no sampling)
def get_tie(n, seed, strats, sample_sets, config):
    '''

    :param n: float, timing n
    :param seed: index of player i, default to none
    :param strats: array, current strategies
    :param sample_sets: ndarray, sample sets (None if sampling is none), give each player an array of random other players to sample
    :param config: dict, dictionary containing simulation parameters
    :return: float, ties for timing n
    '''
    # this is only here to fix rounding comparison issues
    n = round(n,2)

    tie = 0
    samples = []
    # if using sampling, get the strategies of the sampled players
    if (seed is not None) and (config['sampling'] is not None):
        for samp in sample_sets[seed]:
            samples.append(strats[samp])
    # otherwise check all strategies
    else:
        samples = strats
    # compare to strategies to calculate position
    for strat in samples:
        # more rounding stuff due to float precision errors
        strat = round(strat, 2)
        if n == strat:
            tie = tie + 1
    return tie


# Returns the payoff at timing n
def get_y(n, strats, sample_sets, config, seed=None, use_bandwidth=False):
    '''

    :param n: float, timing n
    :param strats: array, current strategies
    :param sample_sets: ndarray, sample sets (None if sampling is none), give each player an array of random other players to sample
    :param config: dict, dictionary containing simulation parameters
    :param seed: index of player i, default to none
    :param use_bandwidth:
    :return: float, payoff
    '''
    # calculate the timing component
    ux = 1 + (2 * config['lambda'] * n) - (n * n)
    ties = get_tie(n, seed, strats, sample_sets, config)
    pos = get_position(n, seed, strats, sample_sets, config)
    if (seed is not None) and (config['purification'] is not None):
        puriVal = 1 - (config['purification'] * seed/len(strats))
    else:
        puriVal = 1
    vy = (1 - (pos * puriVal/len(strats))/config['gamma']) * (1 + (pos * puriVal/len(strats)/config['rho']))
    # if there are ties, calculate the average of the position components over the tie range
    if ties > 0:
        vy = 0
        for j in range(ties):
            vy += (1 - ((pos+j) * puriVal/len(strats))/config['gamma']) * (1 + ((pos+j) * puriVal/len(strats))/config['rho'])
        vy = vy/ties
    # otherwise just use the regular formula
    if (config['bandwidth'] is not None) and use_bandwidth:
        ux = 0
        start_x = n - config['bandwidth']
        for i in range(21):
            ux += 1 + (2 * config['lambda'] * start_x) - (start_x * start_x)
            start_x += config['bandwidth']/10
        # ties = get_tie(n, seed, strats, sample_sets, config)
        # pos = get_position(n, seed, strats, sample_sets, config)
        # vy = (1 - (pos * puriVal/len(strats))/config['gamma']) * (1 + (pos * puriVal/len(strats)/config['rho']))
        # # if there are ties, calculate the average of the position components over the tie range
        # if ties > 0:
        #     vy1 = 0
        #     for j in range(ties):
        #         vy1 += (1 - ((pos+j) * puriVal/len(strats))/config['gamma']) * (1 + ((pos+j) * puriVal/len(strats))/config['rho'])
        #     vy1 = vy/ties
        ux = ux/21
    return ux * vy


def data_logging(var_name_list, round_num, history):
    for var in var_name_list:
        history[var, round_num] = eval({var})

    return history


########## simulation functions ############
import random
import numpy as np
# from timing_games_package import Functions as fun

def initialize_player_strategies(config):
    '''
    Function to run first simulation step to initialize each player's initial strategy
    :param config: dict, dictionary contains simulation parameters
    :return: array, initial strategies;  ndarray, sample sets (None if sampling is none), give each player an array of random other players to sample
    '''
    # game type for starting distribution: set to fear or greed for respective distributions
    # any value other than fear or greed will yield a random start
    game = config['game_type']
    num_bots = config['num_bots']
    lam = config['lambda']
    gam = config['gamma']
    rho = config['rho']
    sampling = config['sampling']
    xmin = config['xmin']
    xmax = config['xmax']

    strategies = []
    sample_sets = []

    # set initial strategies and sampling
    # these calculations are inexact because we have a finite number of players
    # greed, fear, and random starting distributions have differing calculations
    if game == "fear":
        cdfx = np.round(np.arange(config['cdfmin'], config['cdfmax'], 0.01), 2)
        cdfy = gam - rho + np.sqrt((gam + rho) ** 2 - 4 * ((1 + rho) * (gam - 1) * (1 + lam ** 2))/(1 + 2 * lam * cdfx - cdfx ** 2))
        y_ind = 0
        cdfy = cdfy/2
        for i in range(num_bots):
            # y_ind is the index in the cdf to compare to
            # we increment it until it is greater than or equal to the percentage of players set so far
            if (i+1)/num_bots <= cdfy[y_ind]:
                strategies.append(cdfx[y_ind])
            else:
                while y_ind < len(cdfy) - 1 and (i+1)/num_bots > cdfy[y_ind]:
                    y_ind = y_ind + 1
                # there are some rounding issues when we reach the end of the cdf
                # if we reach the end (for the last few players), just use the last value
                if y_ind >= len(cdfy):
                    strategies.append(cdfx[len(cdfy)])
                else:
                    strategies.append(cdfx[y_ind])
            # strategies.append(random.random() * (xmax - xmin) + xmin)
            # give each player a wait time
            # wait_times.append(random.randint(0, wait_time))
            # give each player an array of random other players to sample
            if sampling is not None:
                # to_add = []
                other_player_index_list = list(range(num_bots))
                other_player_index_list.remove(i)
                # for j in range(sampling):
                #     val = random.randint(0, num_bots - 1)
                #     # if we've got this player in the sample set already or player i himself, try again
                #     if val in to_add or val == i:
                #         j = j - 1
                #     else:
                #         to_add.append(val)
                to_add = random.sample(other_player_index_list, sampling)
                sample_sets.append(to_add)
                
    elif game == "greed":
        cdfx = np.round(np.arange(config['cdfmin'], config['cdfmax'], 0.01), 2)
        cdfy = gam - rho - np.sqrt((gam + rho) ** 2 - 4 * gam * rho * (1 + lam ** 2)/(1 + 2 * lam * cdfx - cdfx ** 2))
        y_ind = len(cdfy) - 1
        cdfy = cdfy/2
        i = num_bots
        while i > 0:
            # y_ind is the index in the cdf to compare to
            # we decrement it until it is less than or equal to the percentage of players set so far
            if (i-1)/num_bots >= cdfy[y_ind]:
                strategies.append(cdfx[y_ind])
            else:
                while y_ind > 0 and (i-1)/num_bots < cdfy[y_ind]:
                    y_ind = y_ind - 1
                # there are some rounding issues when we reach the end of the cdf
                # if we reach the end (for the last few players), just use the last value
                if y_ind == 0:
                    strategies.append(cdfx[0])
                else:
                    strategies.append(cdfx[y_ind])
            # strategies.append(random.random() * (xmax - xmin) + xmin)
            # give each player a wait time
            # wait_times.append(random.randint(0, wait_time))
            # give each player an array of random other players to sample
            if sampling is not None:
                # to_add = []
                other_player_index_list = list(range(num_bots))
                other_player_index_list.remove(i)
                # for j in range(sampling):
                #     val = random.randint(0, num_bots - 1)
                #     # if we've got this player in the sample set already or player i himself, try again
                #     if val in to_add or val == i:
                #         j = j - 1
                #     else:
                #         to_add.append(val)
                to_add = random.sample(other_player_index_list, sampling)
                sample_sets.append(to_add)
            i = i - 1
    else:
        for i in range(num_bots):
            strategies.append(random.random() * (xmax - xmin) + xmin)
            # give each player a wait time
            # wait_times.append(random.randint(0, wait_time))
            # give each player an array of random other players to sample
            if sampling is not None:
                # to_add = []
                other_player_index_list = list(range(num_bots))
                other_player_index_list.remove(i)
                # for j in range(sampling):
                #     val = random.randint(0, num_bots - 1)
                #     # if we've got this player in the sample set already or player i himself, try again
                #     if val in to_add or val == i:
                #         j = j - 1
                #     else:
                #         to_add.append(val)
                to_add = random.sample(other_player_index_list, sampling)
                sample_sets.append(to_add)
    strategies = np.round(np.array(strategies), 2)

    return strategies, sample_sets


def calculate_payoff(config, strategies, sample_sets):
    '''

    :param config: dict, dictionary contains simulation parameters
    :param strategies: array, current strategies
    :param sample_sets: ndarray, sample sets (None if sampling is none), give each player an array of random other players to sample
    :return: x: array of possible x; y: array of payoff given timing x; strat_x: sorted strategies from low to high;
             strat_y: array of payoffs given sorted strategies; strategies_y: array of payoffs given strategies
    '''

    lam = config['lambda']
    gam = config['gamma']
    rho = config['rho']
    xmin = config['xmin']
    xmax = config['xmax']
    # set the array of possible x values
    x = np.round(np.arange(xmin, xmax, 0.01),2)
    positions = []
    ties = []
    # set up initial values for landscape positions and ties
    for val in x:
        positions.append(get_position(val, None, strategies, sample_sets, config))
        ties.append(get_tie(val, None, strategies, sample_sets, config))
    positions = np.array(positions)
    # calculate timing component
    ux = 1 + (2 * lam * x) - (x * x)
    vy = []
    quantile = []
    # calculate positional component, including ties
    for i in range(len(positions)):
        if ties[i] == 0:
            vy.append((1 - (positions[i]/len(strategies))/gam) * (1 + (positions[i]/len(strategies))/rho))
            quantile.append(positions[i]/len(strategies))
        else:
            total = 0
            total_quantile = 0
            for j in range(ties[i]):
                total += (1 - ((positions[i]+j)/len(strategies))/gam) * (1 + ((positions[i]+j)/len(strategies))/rho)
                total_quantile += (positions[i])/len(strategies)
            total = total/ties[i]
            vy.append(total)
            total_quantile = total_quantile/ties[i]
            quantile.append(total_quantile)
            
    quantile = np.array(quantile)
    if config['game_type'] == 'fear':
        quantile = quantile - 1/config['num_bots']
    elif config['game_type'] == 'other':
        quantile = quantile - 0.5/config['num_bots']    
        
    y = ux * vy
    strat_x = np.sort(strategies)
    strat_y = []
    strategies_y = []
    # calculate bubble positions
    for strat in strat_x:
        strat_y.append(get_y(strat, strategies, sample_sets, config, seed=None, use_bandwidth=False))
    for strat in strategies:
        strategies_y.append(get_y(strat, strategies, sample_sets, config, seed=None, use_bandwidth=False))
    return x, y, strat_x, strat_y, strategies_y, quantile

# Loops through all players and moves them if they are ready to move
def update_player_strategies(x, y, strategies, strategies_y, sample_sets, config):
    '''

    :param x: numpy array, array of possible x
    :param y: numpy array, payoff array w.r.t each element in x
    :param strategies: list, list containing previous strategies of each player
    :param config: dict, dictionary containing simulation parameters
    :return: strategies: list, list containing updated strategies of each player
    '''

    theta = config['theta']
    asynchronous = config['asynchronous']
    trembling = config['trembling']
    move_size = round(config['move_percent'] *config['num_bots'])

    if asynchronous is True:
        best_possible = max(y)

        # jump frequencies proportional to regret
        distances_from_best_payoff = abs(strategies_y-best_possible)
        players_index = list(range(len(strategies)))
        selected_player_index = random.choices(players_index, weights=distances_from_best_payoff,k=move_size)
        selected_player_index = set(selected_player_index)
        if len(selected_player_index) != move_size:
            remaining_player_index = set(players_index) - selected_player_index
            remaining_selected_index = random.sample(remaining_player_index,k=move_size-len(selected_player_index))
            selected_player_index.update(remaining_selected_index)
            
        for i in selected_player_index:
            selected_player_strategy = strategies[i]
        # find best payoff index
            if config['sampling'] is not None:
                y1 = []
                for val in x:
                    y1.append(get_y(val, strategies, sample_sets, config, seed=i, use_bandwidth=True))
                best = max(y1)
                # if there are multiple timings with the best payoff, choose randomly
                indices = [k for k, j in enumerate(y1) if j == best]
                best_choice = random.choice(indices)
                best_choice = x[best_choice]
                choice_set = np.array([best_choice, selected_player_strategy])
            else:
                indices = [k for k, j in enumerate(y) if j == best_possible]
                best_choice = random.choice(indices)
                best_choice = x[best_choice]
                choice_set = np.array([best_choice, selected_player_strategy])
                
        # moving chance
            if theta is not None:
                choice = random.choices(choice_set, weights=[theta, 1-theta], k=1)

            # chance = theta + 10*(best_possible - selected_player_payoff)/best_possible
            # chance = min(chance, theta+0.05)
            # chance = max(chance, theta-0.05)
            # choice = random.choices(choice_set, weights=[chance, 1-chance], k=1)

        # apply trembling
                strategies[i] = choice[0] + round((random.random() * trembling - trembling/2), 2)
            else:
                strategies[i] = best_choice + round((random.random() * trembling - trembling/2), 2)

    # if all players make move simultaneously
    else:
        selected_player_index = None
    # loop over all players
        static_strats = np.copy(strategies)
        best_possible = max(y)
        for i in range(len(strategies)):
            y1 = []
            for val in x:
                y1.append(get_y(val, static_strats, sample_sets, config, seed=i, use_bandwidth=True))
    #         print(f'{i} needs to compare payoff in y1 {y1}')
            # find the best observed payoff
            best = max(y1)
            # if there are multiple timings with the best payoff, choose randomly
            indices = [k for k, j in enumerate(y1) if j == best]
            best_choice = random.choice(indices)
            best_choice = x[best_choice]
            choice_set = np.array([best_choice, static_strats[i]])
            # moving chance
            if theta is not None:
                choice = random.choices(choice_set, weights=[theta, 1 - theta], k=1)

                # chance = theta + 10*(best_possible - get_y(static_strats[i], static_strats, sample_sets, config, use_bandwidth=True))/best_possible
                # chance = min(chance, theta+0.05)
                # chance = max(chance, theta-0.05)
                # choice = random.choices(choice_set, weights=[chance, 1-chance], k=1)

            # apply trembling
                strategies[i] = choice[0] + round((random.random() * trembling - trembling/2), 2)
            else:
                strategies[i] = best_choice + round((random.random() * trembling - trembling/2), 2)

    return strategies, selected_player_index

# from timing_games_package import Simulation
# from timing_games_package.Configuration import sim_config_init
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
from matplotlib import style
import numpy as np
import pandas as pd
plt.style.use('default')